In [10]:
%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

from IPython.core.debugger import set_trace
import torchvision.models as models
from PIL import Image
from torchvision.transforms import ToTensor,ToPILImage
import pandas as pd
import json
import pickle
import numpy as np
from copy import deepcopy
from torchvision.utils import save_image
from torchvision import transforms
import matplotlib.pyplot as plt
import torch
from flashtorch.activmax import GradientAscent
import flashtorch
from flashtorch.activmax import GradientAscent
from flashtorch.utils import standardize_and_clip,format_for_plotting
from torchvision.datasets import ImageFolder
import torch.nn as nn
from torchvision import datasets
from os.path import join
from pathlib import Path
from shutil import copyfile
import warnings
warnings.simplefilter('ignore')

from Projector import Projector

import sys
import os
%load_ext autoreload
%autoreload 2

sys.path.insert(1, '../lib/pytorch-cnn-visualizations/src/')
from cnn_layer_visualization import CNNLayerVisualization
from layer_activation_with_guided_backprop import GuidedBackprop
from misc_functions import save_gradient_images

from IPython.display import clear_output

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of Projector failed: Traceback (most recent call last):
  File "/home/ymentha/anaconda3/envs/ML/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/ymentha/anaconda3/envs/ML/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/home/ymentha/anaconda3/envs/ML/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/ymentha/anaconda3/envs/ML/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_remov

## Global Variables

In [65]:
# allow to run reduced computations
SINGLE_LAYER = True
#number of layers we compute shit for
N_LAYERS = 0
#number of filters we compute images for
N_REDUCE = 22
#path to the numb image
NUMB_PATH = "../data/tinyimagenet/numb.png"
# how many top pictures we keep
TOP_AVG_SIGN = 4
#number of favourites images per filter
N_FAV = 4
# number of max spikes images per filter
N_FAV_MAX = 3
#where to save/load the saved models
ORIGIN_PATH = "../results/03_04_20"
#where to access the data
DATA_PATH = "../data/"
#where to access the tinyimagenet dataset
TINY_PATH = os.path.join(DATA_PATH,"tinyimagenet/")
#where to access the train images
TRAIN_PATH = os.path.join(TINY_PATH,"train")
#which folder to get the data from
IMGS_PATH = os.path.join(DATA_PATH,"exsmallimagenet")
#where to save the json
SAVE_PATH = "../saved_model/"
#
SAVE_NAME = "03_04_20.json"

## Check labels

In [14]:
import pandas as pd
import os
labels_df = pd.read_csv(os.path.join(TINY_PATH,"labels.txt"), sep=" ", header=None)
labels_df.rename(columns = {0:'id',1:'label',2:'name'},inplace = True)
present_df = pd.DataFrame(os.listdir(TRAIN_PATH))
all_label_present = present_df[present_df[0].isin(labels_df['id'])].shape[0] == present_df.shape[0]
print("All labels are present in the list: {}".format(all_label_present))
labels_df.set_index('id',inplace = True)
dic_label = labels_df.name.to_dict()

All labels are present in the list: True


In [66]:
model_info[0]

{'id': '0',
 'lay': Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 'name': 'Conv2d_0',
 'n_input': 3,
 'n_output': 64,
 'deproj': <Projector.Projector at 0x7f7187a6a3d0>,
 'filters': [{'id': 0,
   'spikes': [1.2020437717437744,
    1.189648985862732,
    1.1791679859161377,
    1.1637085676193237],
   'fav_imgs': ['../data/exsmallimagenet/n03793489/n03793489_1419.JPEG',
    '../data/exsmallimagenet/n04418357/n04418357_7929.JPEG',
    '../data/exsmallimagenet/n04296562/n04296562_6281.JPEG',
    '../data/exsmallimagenet/n07768694/n07768694_1021.JPEG'],
   'grad_path_avg': ['../results/03_04_20/avg_grads/Conv2d_0/0/n03793489_1419_grad.jpg',
    '../results/03_04_20/avg_grads/Conv2d_0/0/n04418357_7929_grad.jpg',
    '../results/03_04_20/avg_grads/Conv2d_0/0/n04296562_6281_grad.jpg',
    '../results/03_04_20/avg_grads/Conv2d_0/0/n07768694_1021_grad.jpg'],
   'max_spikes': [10.061593055725098, 10.027844429016113, 9.31694221496582],
   'max_slices': [[[73, 76], [79, 82]],


## Process Data

In [15]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    allows to iterate with (datas,labels,paths)
    """
    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [16]:
preprocess = transforms.Compose([
    transforms.CenterCrop(224), 
    transforms.ToTensor()
    ,transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
my_dataset = ImageFolderWithPaths(TRAIN_PATH,transform=preprocess)

CLASS2INDX = my_dataset.class_to_idx
train_loader = torch.utils.data.DataLoader(my_dataset, batch_size=3, shuffle=True)

## Model

In [32]:
def build_model(model):
    model_info = []
    layers = []
    #construct the data structure
    for layer in list(model.features.named_children()):
        lay_info = {'id':layer[0],
                  'lay':layer[1],
                  'name':str(layer[1]).split('(')[0] + "_" + str(layer[0]) 
                }
        if (isinstance(layer[1],(nn.Conv2d,nn.MaxPool2d))):
            layers.append(layer[1])
        if (isinstance(lay_info['lay'],nn.Conv2d)):     
            n_input = lay_info['lay'].in_channels
            n_output = lay_info['lay'].out_channels
            lay_info['n_input'] = n_input
            lay_info['n_output'] = n_output
            lay_info['deproj'] = Projector(deepcopy(layers),224)
            lay_info["filters"] = []
            for i in range(n_output):
                lay_info["filters"].append({
                    "id":i,
                    "spikes":[0 for i in range(N_FAV)],
                    "fav_imgs":[NUMB_PATH for i in range(N_FAV)],
                    "grad_path_avg":[NUMB_PATH for i in range(N_FAV)],
                    "max_spikes":[0 for i in range(N_FAV_MAX)],
                    "max_slices":[[[0,0],[0,0]]for i in range(N_FAV_MAX)],
                    "max_imgs":[NUMB_PATH for i in range(N_FAV_MAX)],
                    "grad_path_max":[NUMB_PATH for i in range(N_FAV_MAX)],
                    "actmax_im":NUMB_PATH
                })
        elif (type(lay_info['lay']) == nn.Linear):
                n_input = lay_info['lay'].in_features
                n_output = lay_info['lay'].out_features
                lay_info['n_output'] = n_output
                #lay_info["filters"] = [empty_filter.copy() for i in range(n_output)]
        model_info.append(lay_info)
    return model_info

In [49]:
def save_to_json(model_info,filename=SAVE_NAME):
    model_info2 = deepcopy(model_info)
    for lay_info in model_info2:
        if (isinstance(lay_info['lay'],nn.Conv2d)):
            del lay_info['deproj']
        del lay_info['lay']
    with open(os.path.join(SAVE_PATH,filename), 'w') as fout:
        json.dump(model_info2, fout, indent = 2)
    print("done!") 

In [34]:
def load_from_json(filename = 'test_model.json'):
    layers = []
    with open(os.path.join(SAVE_PATH,filename), 'r') as fin:
        model_info = json.load(fin)
    for lay_info,layer in zip(model_info,model.features):
        lay_info['lay'] = layer
        if (isinstance(layer,(nn.Conv2d,nn.MaxPool2d))):
            layers.append(layer)
        if (isinstance(layer,nn.Conv2d)):
            lay_info['deproj'] = Projector(deepcopy(layers),224)
    print("done!") 
    return model_info

In [36]:
model_info = build_model(model)

In [27]:
model_info = load_from_json('02_04_20.json')

done!


In [59]:
save_to_json(model_info,'test_model.json')

done!


In [26]:
def get_filt_string(dir_type,layer_name,filter_id):
    """
    return the path to the max-activ-image for a given filter
    """
    return os.path.join(ORIGIN_PATH,dir_type,layer_name,str(filter_id))

In [56]:
def extract_name(img_path,ext='.jpg'):
    """
    extract the name of the imgpath and add the extension
    """
    jpg_name = img_path.split("/")[-1]
    img_name = jpg_name.split(".")[0] + ext
    return img_name

## Create directories

In [40]:
def create_folders(direc_types,model_info):
    """
    create the directories to stock the images
    Args:
        origin_path(str): where to create these dirs
        direc_types(list:str): "gradients","max_activ","cropped"
        
    """
    for dirtype in direc_types:
        subpath = os.path.join(ORIGIN_PATH,dirtype)
        Path(subpath).mkdir(parents=False, exist_ok=True)
        for i,lay_info in enumerate(model_info):
            if (type(lay_info['lay']) == nn.Conv2d):
                subpath2 = os.path.join(subpath,lay_info['name'])
                Path(subpath2).mkdir(parents=False, exist_ok=True)
                for j,filt in enumerate(lay_info['filters']):
                    subpath3 = os.path.join(subpath2,str(filt["id"]))
                    Path(subpath3).mkdir(parents=False, exist_ok=True)

In [41]:
create_folders(["avg_grads","max_grads","cropped","cropped_grad","max_activ"],model_info)

## Favorite images

In [45]:
#Watch out which path you give for the IMGS_PATH
my_dataset = ImageFolderWithPaths(IMGS_PATH,transform=preprocess)
CLASS2INDX = my_dataset.class_to_idx
data_loader = torch.utils.data.DataLoader(my_dataset, batch_size=1, shuffle=True)

def sort_filters_spikes(model_info):
    """
    sorts the spikes and respective paths of the filters inplace
    """
    for lay_info in model_info:
        if (isinstance(lay_info['lay'],nn.Conv2d)):
            for filtr in lay_info['filters']:
                max_indx = np.argsort(filtr["max_spikes"])[::-1]
                filtr["max_spikes"] = np.array(filtr["max_spikes"])[max_indx].tolist()
                filtr["max_imgs"] = np.array(filtr["max_imgs"])[max_indx].tolist()
                filtr["max_slices"] = np.array(filtr["max_slices"])[max_indx].tolist()

                avg_indx = np.argsort(filtr["spikes"])[::-1]
                filtr["spikes"] = np.array(filtr["spikes"])[avg_indx].tolist()
                filtr["fav_imgs"] = np.array(filtr["fav_imgs"])[avg_indx].tolist()
                
def update_favims_model(model_info,data_loader,verbose = False):
    """
    update all the layers of the model_info such that each filter of each layer knows what are
    its favourite images (write it down in json)
    """
    for j,(datas,labels,paths) in enumerate(data_loader):
        print("Progression update favimgs:{:.2f} %".format(j/len(data_loader) * 100))
        for i,lay_info in enumerate(model_info):
            clear_output(wait=True)
            if verbose:
                pass
                #print("Layer{}/{}:{:.2f} %..".format(i,len(model_info),100*j/ len(data_loader)))
                
            #datas: Batchsize x Numberfilter x Nout x Nout
            datas = lay_info['lay'](datas)
            if (not isinstance(lay_info['lay'],nn.Conv2d) ):
                continue
            if (i >N_LAYERS and SINGLE_LAYER):
                break

            batch_size = datas.size(0)
            filter_nb = datas.size(1)
            width = datas.size(3)
            
            #set_trace()
            #spikes: Batchsize x Filternumber
            max_spikes,max_pos = datas.view([batch_size,filter_nb,-1]).max(dim = 2)
            max_rows = max_pos / width
            max_cols = max_pos % width
            
            avg_spikes = datas.view([batch_size,filter_nb,-1]).mean(dim = 2)

            update_filters_maxim(lay_info,max_spikes,paths,max_rows,max_cols)
            update_filters_favim(lay_info,avg_spikes,paths,"avg")
            #save the whole model
    sort_filters_spikes(model_info)
    save_cropped(model_info)
    save_to_json(model_info)

def update_filters_maxim(lay_info,batch_spikes,paths,max_rows,max_cols):
    #as many spikes in batch_spikes as there are samples in batch
    for spikes,path,rows,cols in zip(batch_spikes,paths,max_rows,max_cols):
        #at this stage there are as many spike in spikes as there are filters
        for k,(filt,spike,row,col) in enumerate(zip(lay_info["filters"],spikes.detach().numpy(),rows,cols)):
            #compute the minimum spike for the filter
            min_indx = np.argmin(filt["max_spikes"])
            min_spike = min(filt["max_spikes"])
            
            if (spike > min_spike and not (path in filt["max_imgs"])):
                
                ((x1,x2),(y1,y2)) = lay_info["deproj"].chain(((row.item(),row.item()),(col.item(),col.item())))
                assert(isinstance(x1,int) and isinstance(x2,int) and isinstance(y1,int) and isinstance(y2,int))
                filt["max_slices"][min_indx] = ((x1,x2),(y1,y2))
                filt["max_imgs"][min_indx] = path
                filt["max_spikes"][min_indx] = float(spike)
    
def update_filters_favim(lay_info,batch_spikes,paths,func_type):
    #as many spikes in batch_spikes as there are samples in batch
    for spikes,path in zip(batch_spikes,paths):
        #at this stage there are as many spike in spikes as there are filters
        for k,(filt,spike) in enumerate(zip(lay_info["filters"],spikes.detach().numpy())):
            #compute the minimum spike for the filter
            min_indx = np.argmin(filt["spikes"])
            min_spike = min(filt["spikes"])
            if (spike > min_spike and not (path in filt["fav_imgs"])):
                filt["fav_imgs"][min_indx] = path
                filt["spikes"][min_indx] = float(spike)

In [46]:
def save_cropped(model_info,grad = False,verbose=False):
    """
    iterate on the model_info to save a cropped version of the images
    Args:
        grad(Bool): whether to save the gradients versions
    """
    if grad:
        filtrlist = "grad_path_max" 
        folder = "cropped_grad"
    else:
        filtrlist = "max_imgs"
        folder = "cropped"
        
    for i,lay_info in enumerate(model_info):
        clear_output(wait=True)
        if verbose:
            print("Progression:{} %".format(i/len(model_info)*100))
        if (not isinstance(lay_info['lay'],nn.Conv2d)):
            continue
        for filtr in lay_info['filters']:
            for path,slices in zip(filtr[filtrlist],filtr['max_slices']):
                if (path == NUMB_PATH):
                    continue
                cropped = ToTensor()(Image.open(path))
                ((x1,x2),(y1,y2)) = slices
                cropped = cropped[:,x1:x2+1,y1:y2+1]
                crop_path = get_filt_string(folder,lay_info['name'],filtr['id'])
                file_name = path.split('/')[-1].lower()
                file_path = os.path.join(crop_path,file_name)
                ToPILImage()(cropped).save(file_path)

In [47]:
update_favims_model(model_info,data_loader,verbose = True)

TypeError: save_to_json() missing 1 required positional argument: 'filename'

## Compute the max-act-imgs

In [51]:
def compute_imgs_model(model_info):
    """
    compute and save the filter max-activations-images for each layer (Watch out, very expensive)
    """
    for i,lay_info in enumerate(model_info):
        if (type(lay_info['lay']) == nn.Conv2d):
            for j,filt in enumerate(lay_info['filters']):
                path = os.path.join(origin_path,"gradients",lay_info['name'],str(filt["id"]))
                compute_imgs_layer(lay_info)

    for lay_info in model_info:
        if (type(lay_info["lay"]) == nn.Conv2d):
            compute_imgs_layer(lay_info)
    save_to_json(model_info)

            
def compute_filter_actmax(lay_info,indexes = None):
    """
    compute  and save the filter maximal activation as an image. Compute it only for filters for which
    it has not been computed yet: delte the existing image for a refresh.
    """
    if indexes is None:
        indexes = range(lay_info["lay"].out_channels)
    layer_name = lay_info["name"]
    pre_existing = []
    for filt in lay_info["filters"]:
        name = "{}_{}_max_activ.jpg".format(lay_info['name'],filt['id'])
        filt_path = get_filt_string("max_activ",lay_info["name"],filt["id"])
        filt_path = os.path.join(filt_path,name)
        try:
            f = open(filt_path)
            filt["actmax_im"] = filt_path
            pre_existing.append(filt["id"])
            f.close()
        except FileNotFoundError:
            pass
    
    indexes = [i for i in indexes if i not in pre_existing]
    
    for filt_indx in indexes:
        filt = lay_info['filters'][filt_indx]
        visualizer = CNNLayerVisualization(model.features, 
                                           selected_layer=int(lay_info['id']), 
                                           selected_filter=filt_indx)
        act_max_img = visualizer.visualise_layer_with_hooks()
        filt_path = get_filt_string("max_activ",lay_info['name'],filt['id'])
        name = "{}_{}_max_activ.jpg".format(lay_info['name'],filt['id'])
        filt_path = os.path.join(filt_path,name)
        #save the image
        ToPILImage()(act_max_img).save(filt_path)
        filt["actmax_im"] = filt_path

In [52]:
# test for a few filters
compute_filter_actmax(model_info[0],[2,3])
compute_filter_actmax(model_info[2],[2,3])
save_to_json(model_info)

done!


## Compute the gradients for the favourite images

In [57]:
def compute_grads(gbp,model_info,verbose = False,compute_avg = True,compute_max = True):
    """
    compute the gradients for the fav images of all filters of all layers for the model_info
    Args:
        gbp (GuidedBackprop): fitted on the model
        model_info (dic): as described above
        origin_path (str): path where to store the folders containing the gradient images
    """
    #TODO: add model in there
    for i,lay_info in enumerate(model_info):
        if (i > N_LAYERS and SINGLE_LAYER):
            break
        if (isinstance(lay_info['lay'],nn.Conv2d)):
            for j,filt in enumerate(lay_info['filters']):
                clear_output(wait=True)
                print("Grads Progression:layer{}/{} {}%".format(i,len(model_info),j/len(lay_info['filters'])*100))
                if compute_avg:
                    path = get_filt_string("avg_grads",lay_info['name'],filt["id"])
                    compute_grads_filt(gbp,filt,path,lay_info['id'],"fav_imgs")
                if compute_max:
                    path = get_filt_string("max_grads",lay_info['name'],filt["id"])
                    compute_grads_filt(gbp,filt,path,lay_info['id'],"max_imgs")
                    save_cropped(model_info,grad= True)
    save_to_json(model_info)

In [60]:
def compute_grads_filt(gbp,filt,path,lay_id,img_type):
    """
    compute the gradients wrt to the favourite images of a filter filt.
    Args:
        gbp (GuidedBackprop): fitted on the model
        filt (dic): filter from a layer
        path (str): path to the folder where to store the gradient images
        img_type (str): either "fav_imgs" or "max_imgs"
    """
    grad_strindx = "grad_path_avg" if img_type == "fav_imgs" else "grad_path_max"
    
    for i,img_path in enumerate(filt[img_type]):
        if (img_path == NUMB_PATH):
            continue   
            
        #name of the image
        img_name = extract_name(img_path,"_grad.jpg")
        #joined path and imagename
        grad_path = os.path.join(path,img_name)

        try:
            f = open(grad_path)
            filt[grad_strindx][i] = grad_path
            f.close()
            continue
        except FileNotFoundError:
            pass
        
        image = Image.open(img_path)
        image = preprocess(image).unsqueeze(0)
        image.requires_grad = True
        class_name = img_path.split("/")[-1].split("_")[0]
        gradient = gbp.generate_gradients(image,CLASS2INDX[class_name],lay_id,filt['id'])
        #normalization of the gradient
        gradient = gradient - gradient.min()
        gradient /= gradient.max()
        im = ToPILImage()(gradient[0])
        im.save(grad_path)
        filt[grad_strindx][i] = grad_path

In [61]:
compute_grads(gbp,model_info,verbose=True)

done!


In [62]:
save_to_json(model_info)

done!
